In [1]:
from enum import Enum

In [1]:
# 式（1）
def get_L_H_i(_Ldash_H_i, _f_R_Evp_i, _f_R_Eqp_i, _f_TD_i):
    return _Ldash_H_i * _f_R_Evp_i * _f_R_Eqp_i * _f_TD_i


In [2]:
# 式（2）
def get_f_R_evp_i(_Q, _region, _HeatingOperation):
    
    # table
    (a_R_Evp_i, b_R_Evp_i) = {
        'Region1' : {'I' : (0.031, 0.971), 'RO' : (0.041, 0.975), 'HA' : (0.059, 1.038)},
        'Region2' : {'I' : (0.032, 0.966), 'RO' : (0.043, 0.970), 'HA' : (0.060, 1.034)},
        'Region3' : {'I' : (0.030, 0.963), 'RO' : (0.039, 0.970), 'HA' : (0.050, 1.049)},
        'Region4' : {'I' : (0.027, 0.972), 'RO' : (0.033, 0.985), 'HA' : (0.040, 1.081)},
        'Region5' : {'I' : (0.028, 0.966), 'RO' : (0.034, 0.981), 'HA' : (0.038, 1.092)},
        'Region6' : {'I' : (0.029, 0.961), 'RO' : (0.035, 0.974), 'HA' : (0.039, 1.090)},
        'Region7' : {'I' : (0.020, 0.921), 'RO' : (0.024, 0.937), 'HA' : (0.021, 1.094)},
    }[_region][_HeatingOperation]
    
    return a_R_Evp_i * _Q + b_R_Evp_i

#### Example

In [3]:
get_f_R_evp_i(1000, 'Region7', 'HA')

22.094

In [4]:
# 式（3）
def get_f_R_Eqp_i(_R_l_i, _Equipment, _HeatingOperation):
    
    #table
    a_R_Eqp_i = {'I' : -0.105, 'RO' : -0.137, 'HA' : -0.231}[_HeatingOperation]
    
    if _Equipment == 'yukadanbo': # 暫定の名称。
        return a_R_Eqp_i * _R_l_i + 1
    elif _Equipment != 'yukadanbo':
        return 1



####  Example

In [5]:
get_f_R_Eqp_i(10,'yukadanbo', 'I')

-0.050000000000000044

In [6]:
# 式（4）
def f_TD_i(_Q, _Equipment, _HeatingOperation, _region):
    
    (a_TD_i, f_TD_max) = {
        'Region1' : {'I' : (0.0157, 1.0842), 'RO' : (0.0163, 1.0862), 'HA' : (0.0176, 1.0860)},
        'Region2' : {'I' : (0.0157, 1.0928), 'RO' : (0.0163, 1.0954), 'HA' : (0.0176, 1.0981)},
        'Region3' : {'I' : (0.0097, 1.1048), 'RO' : (0.0101, 1.1079), 'HA' : (0.0110, 1.1147)},
        'Region4' : {'I' : (0.0063, 1.1111), 'RO' : (0.0066, 1.1146), 'HA' : (0.0072, 1.1235)},
        'Region5' : {'I' : (0.0045, 1.1223), 'RO' : (0.0047, 1.1264), 'HA' : (0.0053, 1.1391)},
        'Region6' : {'I' : (0.0045, 1.1277), 'RO' : (0.0047, 1.1320), 'HA' : (0.0053, 1.1465)},
        'Region7' : {'I' : (0.0014, 1.1357), 'RO' : (0.0015, 1.1404), 'HA' : (0.0017, 1.1576)},
    }[_region][_HeatingOperation]
    
    if _Equipment == 'yukadanbo': # 暫定の名称
        return 1
    elif _Equipment != 'yukadanbo':
        return min(a_TD_i * pow(_Q,2) + 1, f_TD_max)
    



In [7]:
f_TD_i(_Q=0.5, _Equipment='a', _HeatingOperation='HA', _region='Region7')


1.000425

In [8]:
# 式（5a）
def L_CS_i(_Ldash_CS_i):
    return _Ldash_CS_i


In [9]:
# 式（5b）
def L_CL_i(_Ldash_CL_i):
    return _Ldash_CL_i

In [4]:
# 式（6） ※12個の居室を配列で計算するか。
# 入力は主居室、その他居室、全体となる。これをどこで12個に分解するか。
def get_L_dash_H_i(_L_dash_H_R_i, _A_HCZ_i, _A_HCZ_R_i):
    return _L_dash_H_R_i * _A_HCZ_i / _A_HCZ_R_i

def get_L_dash_CS_i(_L_dash_CS_R_i, _A_HCZ_i, _A_HCZ_R_i):
    return _L_dash_CS_R_i * _A_HCZ_i / _A_HCZ_R_i

def get_L_dash_CL_i(L_dash_CL_R_i, A_HCZ_i, A_HCZ_R_i):
    return L_dash_CL_R_i * A_HCZ_i / A_HCZ_R_i

#### Example

In [7]:
print(get_L_dash_H_i(10, 20, 29.81))
print(get_L_dash_CS_i(20, 50, 51.34))
print(get_L_dash_CL_i(30, 10, 20))

6.709158000670916
19.477989871445267
15.0


In [2]:
class Heat_Storage(Enum):
    Exist = 1,
    NotExist = 2

In [3]:
# 式（7）
# WEBプログラムの実装状況によっては修正が始まる。
def get_L_dash_H_R_i(_HeatStorage, L_dash_H_R_TS_0_i, L_dash_H_R_TS_1_i, _delta_L_dash_H_uf_R_i):
    if _HeatStorage == Heat_Storage.NotExist or L_dash_H_R_TS_0_i <= L_dash_H_R_TS_1_i:
        return L_dash_H_R_TS_0_i - _delta_L_dash_H_uf_R_i
    elif _heatStorage == Heat_Storage.Exist and L_dash_H_R_TS_0_i > L_dash_H_R_TS_1_i:
        return L_dash_H_R_TS_1_i - _delta_L_dash_H_uf_R_i
        

In [9]:
# 式（8）
def L_dash_H_R_TSl_i(_Q_HEXC, person):
    
    L_Q1 = 1
    
    
    (_Q_HEXC - _Q2) / (_Q1 - _Q2) * L_Q1 + (_Q_HEXC - _Q1) * L_Q2 / (_Q2 - _Q1)
    